<a href="https://colab.research.google.com/github/ayushkalani/logic-dataset-creation-nlp/blob/master/NLPDatasetCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!python --version

Mounted at /content/drive
Python 3.7.12


In [28]:
# Wordnet
!pip install names
# import nltk
# nltk.download('wordnet')
# from nltk.corpus import wordnet as wn

# lem = wn.lemmas('frustration')
# print(lem)
# NOUNS = [word for synset in wn.all_synsets('n') for word in synset.lemma_names()]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[Lemma('frustration.n.01.frustration'), Lemma('frustration.n.02.frustration'), Lemma('frustration.n.03.frustration')]


In [3]:
relations = ['RelatedTo','IsA','FormOf','PartOf','SimilarTo','HasA','UsedFor','CapableOf','AtLocation','Causes','HasSubevent','HasPrerequisite','HasProperty','MotivatedByGoal','ObstructedBy','Desires','CreatedBy','Synonym','DistinctFrom','DerivedFrom','SymbolOf','DefinedAs','MannerOf','LocatedNear','HasContext','EtymologicallyRelatedTo','EtymologicallyDerivedFrom','CausesDesire','MadeOf','ReceivesAction']

In [59]:
import names

def create_premise(klass, *sentences):
  premise = ""
  for s in sentences:
    premise += (s + ". ")
  name = names.get_first_name()
  inference = "{} is a {}".format(name, klass)
  #premise += (". "+ inference)
  premise += inference
  return [premise, name]

def get_qa(questions, ans):
  return {"Questions": questions, "Answers": [ans] * len(questions) }

def create_question_type_1(questionable_name, word):
  return "Does {} has {}?".format(questionable_name, word)

def create_question_type_2(questionable_name, word):
  return "Does {} not has {}?".format(questionable_name, word)

def create_question_type_3(questionable_name, word_1, word_2, operator):
  return "Does {} has {} {} {}?".format(questionable_name, word_1, operator, word_2)

# does john has tail?
def create_question_type_4(questionable_name, word):
   antonym = get_antonym_of_word(word)
   if antonym:
      return "Does {} has {}?".format(questionable_name, antonym)
   else:
     return None

def get_synonym_of_word(klass):
  url = 'http://api.conceptnet.io/query?end=/c/en/{}&rel=/r/Synonym&limit=1000'.format(klass)
  obj = requests.get(url).json()
  # return the first antonym
  for edge in obj['edges']:
    return (edge['start']['label'] or '')     

def get_antonym_of_word(klass):
  url = 'http://api.conceptnet.io/query?end=/c/en/{}&rel=/r/Antonym&limit=1000&filter=/c/en'.format(klass)
  obj = requests.get(url).json()
  # return the first antonym
  for edge in obj['edges']:
    return (edge['start']['label'] or '')

In [49]:
# Conjunction
import requests, re, traceback
import pandas as pd
pattern = r"[\([{})\]]"
OPERATORS = ["and", "or"]
YES ="yes"
NO = "no"

def mapping_for_conjunction(klass):
  url = 'http://api.conceptnet.io/query?start=/c/en/{}&rel=/r/HasA&limit=1000'.format(klass)
  obj = requests.get(url).json()
  dataum = {}
  df_klass = pd.DataFrame()
  for edge in obj['edges']:
    if edge['surfaceText']:
      dataum[edge['end']['label']] = re.sub(pattern, "", edge['surfaceText'])
  
  temp = list(dataum)
  result = {'Questions': [], 'Answers': []}
  for i in range(0, len(temp)):
    word_1 = temp[i]
    for j in range(i+1, len(temp)):
      try:
          word_2 = temp[j]
          sent_1 = dataum[word_1]
          sent_2 = dataum[word_2]
          premise, questionable_name = create_premise(klass, sent_1, sent_2)
          questions_type1 = []; questions_type2 = []; questions_type3 = []; questions_type4 = []
          # does john has tail?
          for w in [word_1, word_2]:
            questions_type1.append(create_question_type_1(questionable_name, w))
            questions_type2.append(create_question_type_2(questionable_name, w))
            questions_type4.append(create_question_type_4(questionable_name, w))
          # does johhn has tail or fur?
          # does johhn has tail and fur?
          for oprand in OPERATORS:
            questions_type3.append(create_question_type_3(questionable_name, word_1, word_2, oprand))
          questions_type4_final_list = list(filter(None, questions_type4))
          q1=get_qa(questions_type1, YES)
          q2=get_qa(questions_type2, NO)
          q3=get_qa(questions_type3, YES)
          q4=get_qa(questions_type4_final_list, NO)
          result['Rules'] = 'Conjunction'
          result['Context'] = premise
          result['Questions'] = q1['Questions'] + q2['Questions'] + q3['Questions'] + q4['Questions']
          result['Answers'] = q1['Answers'] + q2['Answers'] + q3['Answers'] + q4['Answers']
          df_klass = pd.concat([df_klass,pd.DataFrame(result)],ignore_index=True)
      except Exception as e:
        print(e)
        print(traceback.format_exc())
        return df_klass
  return df_klass

In [ ]:
from uuid import uuid4
df = pd.DataFrame(columns=['UUID','Rules','Context','Questions','Answers'])
current_words = ["countries", "animals", "cities", "sports", "cultures", "dogs", "cats", "buildings", "colleges", "subjects", "lakes", "mountains", "dishes"]

for noun in current_words:
  try:
    dn = mapping_for_conjunction(noun)
    df = pd.concat([df, pd.DataFrame(dn)], ignore_index=True)
  except Exception as e:
    print(e)
    print(traceback.format_exc())
    continue
df['UUID'] = df.index.to_series().map(lambda x: uuid4())
df.to_csv("Dataset5_AyushKalani.csv", sep=',', index=False)

In [ ]:
len(df)
display(df.head())

### ADdDITON
**ADDITIOn**

In [192]:
import inflect
inflect = inflect.engine()

def is_word_plural(w):
  try:
    return not inflect.singular_noun(w) == False
  except Exception as e:
    return False

def create_addition_question_type_1(questionable_name, entity):
  url = 'https://api.conceptnet.io/query?start=/c/en/{}&rel=/r/RelatedTo'.format(entity)
  obj = requests.get(url).json()
  questions = []

  for edge in obj['edges']:
    questions.append("Is {} a {} or {}".format(questionable_name, entity, edge['end']['label']))
    if (not is_word_plural(entity)) and (not is_word_plural(edge['end']['label'])):
      questions.append("Is {} a {} or not a {}".format(questionable_name, entity, edge['end']['label']))
  return questions

def create_addition_question_type_2(questionable_name, entity):
  return "Is {} not a {}?".format(questionable_name, entity)

In [193]:
# Addition
import requests, re, traceback
import pandas as pd
pattern = r"[\([{})\]]"
OPERATORS = ["and", "or"]
YES ="yes"
NO = "no"

def mapping_for_addition(klass, relation):
  url = 'http://api.conceptnet.io/query?start=/c/en/{}&rel=/r/{}&limit=1000'.format(klass, relation)
  obj = requests.get(url).json()
  dataum = {}
  df_klass = pd.DataFrame()
  for edge in obj['edges']:
    if edge['surfaceText']:
      dataum[edge['end']['label']] = re.sub(pattern, "", edge['surfaceText'])
  
  temp = list(dataum)
  result = {'Questions': [], 'Answers': []}
  for i in range(0, len(temp)):
    try:
        word_1 = temp[i]
        premise, questionable_name = create_premise(klass, dataum[word_1])
        questions_type1 = []; questions_type2 = []
        questions_type1 = create_addition_question_type_1(questionable_name, word_1)
        questions_type2.append(create_addition_question_type_2(questionable_name, word_1))
        q1=get_qa(questions_type1, YES)
        q2=get_qa(questions_type2, NO)
        result['Rules'] = 'Addition'
        result['Context'] = premise
        result['Questions'] = q1['Questions'] + q2['Questions']
        result['Answers'] = q1['Answers'] + q2['Answers']
        df_klass = pd.concat([df_klass,pd.DataFrame(result)],ignore_index=True)
    except Exception as e:
      print(e)
      print(traceback.format_exc())
      return df_klass
  return df_klass

In [194]:
from uuid import uuid4
df = pd.DataFrame(columns=['UUID','Rules','Context','Questions','Answers'])
current_words = ["countries", "animals", "cities", "sports", "cultures", "dogs", "cats", "buildings", "colleges", "subjects", "lakes", "mountains", "dishes"]

for noun in current_words:
  try:
    dn = mapping_for_addition(noun, "IsA")
    df = pd.concat([df, pd.DataFrame(dn)], ignore_index=True)
  except Exception as e:
    print(e)
    print(traceback.format_exc())
    continue
df['UUID'] = df.index.to_series().map(lambda x: uuid4())
df.to_csv("Dataset5_AyushKalani.csv", sep=',', index=False)